In [0]:
%run /Workspace/Users/dataeng.victor_outlook.com#ext#@dataengvictoroutlook.onmicrosoft.com/01.config/00.global_config


In [0]:
%run /Workspace/Repos/dataeng.victor@outlook.com/databricks-weather-realtime/04.gold/02_gold_snapshot

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

# Caminho da tabela silver
silver_table = f"{catalog}.{silver['schema']}.{silver['table']}"
df = spark.read.table(silver_table)

# Criar window ordenando pela timestamp mais recente
window = Window.orderBy(col("timestamp").desc())

# Selecionar apenas a última leitura
df_snapshot = (
    df.withColumn("rn", row_number().over(window))
      .filter(col("rn") == 1)
      .drop("rn")
)

# Definir nome da tabela snapshot
gold["table_snapshot"] = "weather_latest_snapshot"

# Escrever a tabela delta
# Solução: Adicionar a option("overwriteSchema", "true")
(df_snapshot.write
    .mode("overwrite")
    .format("delta")
    .option("overwriteSchema", "true") 
    .saveAsTable(f"{catalog}.gold.{gold['table_snapshot']}")
)


In [0]:
%sql
SELECT * FROM weather_realtime.gold.weather_latest_snapshot;
